# Imports

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

from HANKStickyModel import HANKStickyModelClass

TypeError: Cannot create a consistent method resolution
order (MRO) for bases object, GEModelClass, HANKStickyAnalyticsClass

# Solve the model

In [5]:
def solution_routine(model, do_print=False):
    """ solve model """
    # init model
    print('\r'+f' -------- Model: {model.name} ---------')
    # solve model
    print("\r"+'Find steady state  ',end="")
    model.find_ss(do_print=do_print)
    print("\r"+'Compute Jacobians  ',end="")
    model.compute_jacs(do_print=do_print)
    print("\r"+'Find transition path',end="")
    model.find_transition_path(do_print=do_print, do_print_unknowns=False)
    print("\r"+'Find IRFs           ',end="")
    model.find_IRFs(do_print=do_print)
    print("\r"+'Done                ',end="")
    # return model


In [3]:
model = HANKStickyModelClass(name='baseline')

In [4]:
par = model.par
ss = model.ss
path = model.path
sim = model.sim

In [6]:
solution_routine(model)

 -------- Model: baseline ---------
Find IRFs           

# Testing

In [72]:
test = HANKStickyModelClass(name="test")
HANKStickyModelClass.mro()

[HANKStickyModel.HANKStickyModelClass,
 EconModel.EconModel.EconModelClass,
 GEModelTools.GEModelClass.GEModelClass,
 AnalyticsClass.HANKStickyAnalyticsClass,
 object]

In [34]:
solution_routine(test)

 -------- Model: test ---------
Find IRFs           

In [41]:
test.calc_MPC(annual=False)

mean MPC out of liquid assets: 0.354 (annual terms)
test


In [45]:
test.jac_hh['C_hh', 'Z']

array([[3.36080077e-01, 3.70927483e-02, 3.28745618e-02, ...,
        1.00590647e-07, 9.55979740e-08, 9.08517706e-08],
       [3.62352866e-02, 3.30162297e-01, 3.32208968e-02, ...,
        1.03109918e-07, 9.79931904e-08, 9.31293973e-08],
       [3.25785053e-02, 3.23145780e-02, 3.26964421e-01, ...,
        1.05832024e-07, 1.00578133e-07, 9.55870296e-08],
       ...,
       [3.36664146e-07, 3.43117255e-07, 3.50250957e-07, ...,
        3.12640716e-01, 1.76469970e-02, 1.62873352e-02],
       [3.21561986e-07, 3.27725574e-07, 3.34539220e-07, ...,
        1.67829032e-02, 3.12640716e-01, 1.76469970e-02],
       [3.07137309e-07, 3.13024367e-07, 3.19532319e-07, ...,
        1.58981341e-02, 1.67829032e-02, 3.12640716e-01]])

In [47]:
sum([test.jac_hh['C_hh', 'Z'][t, 0] for t in range(4)])

0.4347497734700018

In [46]:
sum([t for t in range(4)])

6

In [44]:
ss_y = ss.Z + (1+ss.rl)*ss.L_hh #+ ss.ra*ss.A_hh
frac_z = ss.Z/ss_y
frac_l = ss.L_hh/ss_y
frac_z * test.jac_hh['C_hh', 'Z'][0,0] + frac_z * test.jac_hh['C_hh', 'Z'][0,0]

1.627049966149837

In [48]:
# for liquid assets
ss = test.ss

MPC = np.zeros(ss.D.shape)
dc = (ss.c[:, :, 1:, :] - ss.c[:, :, :-1, :])
l_grid_full = np.repeat(par.l_grid, par.Na).reshape(1,1,par.Nl,par.Na)
dl = (1 + ss.rl) * l_grid_full[:, :, 1:, :] - (1 + ss.rl) * l_grid_full[:, :, :-1, :]
MPC[:, :, :-1, :] = dc / dl
MPC[:, :, -1, :] = MPC[:, :, -2, :]  # assuming constant MPC at end
mean_MPC = np.sum(MPC * ss.D)
mean_MPC_ann = 1 - (1 - mean_MPC) ** 4
print(f'mean MPC: {mean_MPC:.3f} [annual: {mean_MPC_ann:.3f}]')

mean MPC: 0.354 [annual: 0.826]


In [50]:
(1/(1+ss.r))**3

0.964068794694323

In [55]:
test.path.eg[0,0] = 0.1

In [59]:
test.jac_hh['C_hh', 'Z'].any()

True

In [69]:
np.max(abs(test.jac[('C_hh', 'eg')] - test2.jac[('C_hh', 'eg')]))

0.053772812335095566

In [62]:
test2 = HANKStickyModelClass(name="test")
test2.jac_hh['C_hh', 'Z']

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# Plotting

In [7]:
# paths = ['G','Y','C_hh']
# lvl_value = ['eg', 'em', 'i', 'Pi', 'Pi_w','clearing_Y']
# abs_diff = ['r', 'ra', 'rl', 'i']
# facs = {'r':100,'ra':100,'rl':100}
# model.show_IRFs(paths,lvl_value=lvl_value,abs_diff=abs_diff,facs=facs,ncols=3,T_max=par.T,do_linear=True,do_shocks=True)

# Misc

In [8]:
# model.print_unpack_varlist()

In [9]:
# model.check_non_lin(0)

In [10]:
# print(np.max(path.A - path.A_hh))